In [2]:
!pip install transformers timm


  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.50.3-py3-none-any.whl (10.2 MB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)


In [4]:
# Notebook: jersey_detection.ipynb

import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
from transformers import AutoProcessor, VisionEncoderDecoderModel
import torchvision.transforms as T
import torch.nn.functional as F
import sys
sys.path.append("Real-ESRGAN")  # Real-ESRGAN klasörünü yol olarak ekle
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

# === Adım 0: Yol ayarları ===
image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")
output_path = Path("output/results_frame1.jpg")
yolo_weights = "weights/yolov5s.pt"
model_path = "Real-ESRGAN/weights/RealESRGAN_x4plus.pth"

# === Adım 1: YOLOv5 ile oyuncu tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25  # Tespit güven eşiği

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Adım 2: Torso Crop ===
crops = []
boxes = []
for det in preds:
    x1, y1, x2, y2, conf, cls = det
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    torso_y1 = y1 + int((y2 - y1) * 0.4)
    crop = img[torso_y1:y2, x1:x2]
    if crop.size > 0:
        crops.append(crop)
        boxes.append((x1, torso_y1, x2, y2))

# === Adım 3: Super Resolution (Real-ESRGAN) ===
model_rrdb = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                     num_block=23, num_grow_ch=32, scale=4)
upscaler = RealESRGANer(
    scale=4,
    model_path=model_path,
    model=model_rrdb,
    tile=0,
    tile_pad=10,
    pre_pad=0,
    half=True,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

sr_crops = []
for crop in crops:
    img_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)).convert("RGB")
    output, _ = upscaler.enhance(np.array(img_pil), outscale=1)
    sr_crop = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    sr_crops.append(sr_crop)

# === Adım 4: OCR - EasyOCR, Tesseract, PARSeq ===
reader = easyocr.Reader(["en"], gpu=True)
parseq_model = VisionEncoderDecoderModel.from_pretrained("impira/layoutlm-parseq")
parseq_processor = AutoProcessor.from_pretrained("impira/layoutlm-parseq")
parseq_model.eval()

ocr_results = []
transform = T.Compose([
    T.Resize((384, 384)),
    T.ToTensor(),
    T.Normalize([0.5], [0.5])
])

def run_parseq(img_crop):
    pil = Image.fromarray(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)).convert("RGB")
    inputs = parseq_processor(images=pil, return_tensors="pt")
    with torch.no_grad():
        outputs = parseq_model.generate(**inputs, max_length=4)
    text = parseq_processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return text

for crop in sr_crops:
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    easy_out = reader.readtext(np.array(crop))
    easy_text = easy_out[0][1] if easy_out else ""

    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 7 -c tessedit_char_whitelist=0123456789')
    tess_text = tess_text.strip()

    parseq_text = run_parseq(crop)

    candidates = [easy_text.strip(), tess_text, parseq_text.strip()]
    digits = [c for c in candidates if c.isdigit()]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)

# === Adım 5: Sonuçları görselleştirme ===
for box, number in zip(boxes, ocr_results):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"Çıktı kaydedildi: {output_path}")

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti (Super Resolution Dahil)")
plt.show()



ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

In [3]:
!pip install --upgrade torchvision


In [5]:
# Notebook: jersey_detection.ipynb

import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
from transformers import AutoProcessor, VisionEncoderDecoderModel
import torchvision.transforms as T
import os

# === Adım 0: Yol ayarları ===
image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")
output_path = Path("output/results_frame1.jpg")
crop_dir = Path("crops")
sr_crop_dir = Path("sr_crops")
yolo_weights = "weights/yolov5s.pt"

crop_dir.mkdir(exist_ok=True)
sr_crop_dir.mkdir(exist_ok=True)

# === Adım 1: YOLOv5 ile oyuncu tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Adım 2: Torso Crop ve PNG olarak kaydet ===
boxes = []
for i, det in enumerate(preds):
    x1, y1, x2, y2, conf, cls = det
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    torso_y1 = y1 + int((y2 - y1) * 0.4)
    crop = img[torso_y1:y2, x1:x2]
    if crop.size > 0:
        crop_path = crop_dir / f"crop_{i}.png"
        cv2.imwrite(str(crop_path), crop)
        boxes.append((x1, torso_y1, x2, y2))

# === Adım 3: Super Resolution dışarıda çalıştırılmalı ===
print("🔁 Lütfen 'crops/' klasöründeki görselleri Real-ESRGAN ile netleştirip 'sr_crops/' klasörüne yazın.")
input("Devam etmek için ENTER'a basın...")

# === Adım 4: OCR - EasyOCR, Tesseract, PARSeq ===
reader = easyocr.Reader(["en"], gpu=True)
parseq_model = VisionEncoderDecoderModel.from_pretrained("impira/layoutlm-parseq")
parseq_processor = AutoProcessor.from_pretrained("impira/layoutlm-parseq")
parseq_model.eval()

ocr_results = []

def run_parseq(img_crop):
    pil = Image.fromarray(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)).convert("RGB")
    inputs = parseq_processor(images=pil, return_tensors="pt")
    with torch.no_grad():
        outputs = parseq_model.generate(**inputs, max_length=4)
    text = parseq_processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return text

# OCR için netleştirilmiş kırpılmış görselleri oku
for i, box in enumerate(boxes):
    sr_crop_path = sr_crop_dir / f"crop_{i}.png"
    if not sr_crop_path.exists():
        ocr_results.append("?")
        continue

    crop = cv2.imread(str(sr_crop_path))
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    easy_out = reader.readtext(np.array(crop))
    easy_text = easy_out[0][1] if easy_out else ""

    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 7 -c tessedit_char_whitelist=0123456789')
    tess_text = tess_text.strip()

    parseq_text = run_parseq(crop)

    candidates = [easy_text.strip(), tess_text, parseq_text.strip()]
    digits = [c for c in candidates if c.isdigit()]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)

# === Adım 5: Sonuçları görselleştirme ===
for box, number in zip(boxes, ocr_results):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"Çıktı kaydedildi: {output_path}")

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti (Netleştirilmiş)")
plt.show()


Using cache found in C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-25 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14.1M/14.1M [00:01<00:00, 10.3MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


🔁 Lütfen 'crops/' klasöründeki görselleri Real-ESRGAN ile netleştirip 'sr_crops/' klasörüne yazın.


Devam etmek için ENTER'a basın... 


OSError: impira/layoutlm-parseq is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [6]:
# Notebook: jersey_detection.ipynb

import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
import os

# === Adım 0: Yol ayarları ===

output_path = Path("output/results_frame1.jpg")
crop_dir = Path("crops")
sr_crop_dir = Path("sr_crops")
yolo_weights = "weights/yolov5s.pt"

crop_dir.mkdir(exist_ok=True)
sr_crop_dir.mkdir(exist_ok=True)

# === Adım 1: YOLOv5 ile oyuncu tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25
image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Adım 2: Torso Crop ve PNG olarak kaydet ===
boxes = []
for i, det in enumerate(preds):
    x1, y1, x2, y2, conf, cls = det
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    torso_y1 = y1 + int((y2 - y1) * 0.4)
    crop = img[torso_y1:y2, x1:x2]
    if crop.size > 0:
        crop_path = crop_dir / f"crop_{i}.png"
        cv2.imwrite(str(crop_path), crop)
        boxes.append((x1, torso_y1, x2, y2))

# === Adım 3: Super Resolution dışarıda çalıştırılmalı ===
print("🔁 Lütfen 'crops/' klasöründeki görselleri Real-ESRGAN ile netleştirip 'sr_crops/' klasörüne yazın.")
input("Devam etmek için ENTER'a basın...")

# === Adım 4: OCR - EasyOCR ve Tesseract ===
reader = easyocr.Reader(["en"], gpu=True)
ocr_results = []

# OCR için netleştirilmiş kırpılmış görselleri oku
for i, box in enumerate(boxes):
    sr_crop_path = sr_crop_dir / f"crop_{i}.png"
    if not sr_crop_path.exists():
        ocr_results.append("?")
        continue

    crop = cv2.imread(str(sr_crop_path))
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    easy_out = reader.readtext(np.array(crop))
    easy_text = easy_out[0][1] if easy_out else ""

    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 7 -c tessedit_char_whitelist=0123456789')
    tess_text = tess_text.strip()

    candidates = [easy_text.strip(), tess_text]
    digits = [c for c in candidates if c.isdigit()]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)

# === Adım 5: Sonuçları görselleştirme ===
for box, number in zip(boxes, ocr_results):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"Çıktı kaydedildi: {output_path}")

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti (Netleştirilmiş)")
plt.show()


Using cache found in C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-25 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


🔁 Lütfen 'crops/' klasöründeki görselleri Real-ESRGAN ile netleştirip 'sr_crops/' klasörüne yazın.


Devam etmek için ENTER'a basın... 


Çıktı kaydedildi: output\results_frame1.jpg


In [7]:
# Notebook: jersey_detection.ipynb

import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
import os

# === Adım 0: Yol ayarları ===
image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")
output_path = Path("output/results_frame2.jpg")
crop_dir = Path("crops")
sr_crop_dir = Path("sr_crops")
yolo_weights = "weights/yolov5s.pt"

crop_dir.mkdir(exist_ok=True)
sr_crop_dir.mkdir(exist_ok=True)

# === Adım 1: YOLOv5 ile oyuncu tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Adım 2: Üst Gövde Crop ve PNG olarak kaydet ===
boxes = []
for i, det in enumerate(preds):
    x1, y1, x2, y2, conf, cls = det
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    torso_y2 = y1 + int((y2 - y1) * 0.6)  # Üst %60'lık bölüm
    crop = img[y1:torso_y2, x1:x2]
    if crop.size > 0:
        crop_path = crop_dir / f"crop_{i}.png"
        cv2.imwrite(str(crop_path), crop)
        boxes.append((x1, y1, x2, torso_y2))

# === Adım 3: Super Resolution dışarıda çalıştırılmalı ===
print("🔁 Lütfen 'crops/' klasöründeki görselleri Real-ESRGAN ile netleştirip 'sr_crops/' klasörüne yazın.")
input("Devam etmek için ENTER'a basın...")

# === Adım 4: OCR - EasyOCR ve Tesseract ===
reader = easyocr.Reader(["en"], gpu=True)
ocr_results = []

# OCR için netleştirilmiş kırpılmış görselleri oku
for i, box in enumerate(boxes):
    sr_crop_path = sr_crop_dir / f"crop_{i}.png"
    if not sr_crop_path.exists():
        ocr_results.append("?")
        continue

    crop = cv2.imread(str(sr_crop_path))
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    easy_out = reader.readtext(np.array(crop))
    easy_text = easy_out[0][1] if easy_out else ""

    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 7 -c tessedit_char_whitelist=0123456789')
    tess_text = tess_text.strip()

    candidates = [easy_text.strip(), tess_text]
    digits = [c for c in candidates if c.isdigit()]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)

# === Adım 5: Sonuçları görselleştirme ===
for box, number in zip(boxes, ocr_results):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"Çıktı kaydedildi: {output_path}")

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti (Üst Gövde Crop)")
plt.show()


Using cache found in C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-25 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
WARNING  NMS time limit 0.550s exceeded


🔁 Lütfen 'crops/' klasöründeki görselleri Real-ESRGAN ile netleştirip 'sr_crops/' klasörüne yazın.


Devam etmek için ENTER'a basın... 


Çıktı kaydedildi: output\results_frame2.jpg


In [8]:
import subprocess

subprocess.run([
    "python", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", "crops",
    "-o", "sr_crops"
])


CompletedProcess(args=['python', 'Real-ESRGAN/inference_realesrgan.py', '-n', 'RealESRGAN_x4plus', '-i', 'crops', '-o', 'sr_crops'], returncode=1)

In [9]:
!python inference_realesrgan.py -n RealESRGAN_x4plus -i crops -o sr_crops


python: can't open file 'C:\Users\uysal\inference_realesrgan.py': [Errno 2] No such file or directory


In [24]:
import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
import os

# === Adım 0: Yol ayarları ===

image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")
crop_dir = Path("crops")
output_path = Path("output/results_frame1.jpg")
output_path.parent.mkdir(exist_ok=True)

sr_crop_dir = Path("sr_crops")
yolo_weights = "weights/yolov5s.pt"

crop_dir.mkdir(exist_ok=True)
sr_crop_dir.mkdir(exist_ok=True)

# === Adım 1: YOLOv5 ile oyuncu tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Adım 2: Üst Gövde Crop ve PNG olarak kaydet ===
boxes = []
for i, det in enumerate(preds):
    x1, y1, x2, y2, conf, cls = det
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    torso_y2 = y1 + int((y2 - y1) * 0.6)  # Üst %60'lık bölüm
    crop = img[y1:torso_y2, x1:x2]
    if crop.size > 0:
        crop_path = crop_dir / f"crop_{i}.png"
        cv2.imwrite(str(crop_path), crop)
        boxes.append((x1, y1, x2, torso_y2))

# === Adım 3: Super Resolution - Notebook içinden çalıştırılıyor ===
import subprocess

print("🔁 Real-ESRGAN ile netleştirme başlatılıyor...")
subprocess.run([
    "python", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", "crops",
    "-o", "sr_crops"
])
print("✅ Netleştirme tamamlandı. OCR işlemine geçiliyor...")

# === Adım 4: OCR - EasyOCR ve Tesseract ===
reader = easyocr.Reader(["en"], gpu=True)
ocr_results = []

# OCR için netleştirilmiş kırpılmış görselleri oku
for i, box in enumerate(boxes):
    sr_crop_path = sr_crop_dir / f"crop_{i}.png"
    if not sr_crop_path.exists():
        ocr_results.append("?")
        continue

    crop = cv2.imread(str(sr_crop_path))
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    easy_out = reader.readtext(np.array(crop))
    easy_text = easy_out[0][1] if easy_out else ""

    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 7 -c tessedit_char_whitelist=0123456789')
    tess_text = tess_text.strip()

    candidates = [easy_text.strip(), tess_text]
    digits = [c for c in candidates if c.isdigit()]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)

# === Adım 5: Sonuçları görselleştirme ===
for box, number in zip(boxes, ocr_results):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"Çıktı kaydedildi: {output_path}")

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti (Üst Gövde Crop)")
plt.show()


Using cache found in C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-25 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


🔁 Real-ESRGAN ile netleştirme başlatılıyor...
✅ Netleştirme tamamlandı. OCR işlemine geçiliyor...
Çıktı kaydedildi: output\results_frame1.jpg


In [31]:

import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
import os
import subprocess
import pandas as pd

# === Adım 0: Yol ayarları ===
image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")
crop_dir = Path("crops")
sr_crop_dir = Path("sr_crops")
output_path = Path("output/results_frame1.jpg")
output_path.parent.mkdir(exist_ok=True)
yolo_weights = "weights/yolov5s.pt"

crop_dir.mkdir(exist_ok=True)
sr_crop_dir.mkdir(exist_ok=True)

# === Adım 1: YOLOv5 ile oyuncu tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Adım 2: Üst Gövde Crop ve PNG olarak kaydet ===
boxes = []
for i, det in enumerate(preds):
    x1, y1, x2, y2, conf, cls = det
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    torso_y2 = y1 + int((y2 - y1) * 0.6)  # Üst %60'lık bölüm
    crop = img[y1:torso_y2, x1:x2]
    if crop.size > 0:
        crop_path = crop_dir / f"crop_{i}.png"
        cv2.imwrite(str(crop_path), crop)
        boxes.append((x1, y1, x2, torso_y2))

# === Adım 3: Super Resolution - Notebook içinden çalıştırılıyor ===
print("🔁 Real-ESRGAN ile netleştirme başlatılıyor...")
subprocess.run([
    "python", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", "crops",
    "-o", "sr_crops",
    "--fp32"  # GPU varsa yine kullanılır, CPU fallback için gerekli
])
print("✅ Netleştirme tamamlandı. OCR işlemine geçiliyor...")

# === Adım 4: OCR - EasyOCR ve Tesseract ===
reader = easyocr.Reader(["en"], gpu=True)
ocr_results = []

for i, box in enumerate(boxes):
    sr_crop_path = sr_crop_dir / f"crop_{i}.png"
    if not sr_crop_path.exists():
        ocr_results.append("?")
        continue

    crop = cv2.imread(str(sr_crop_path))
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    easy_out = reader.readtext(np.array(crop))
    easy_text = easy_out[0][1] if easy_out else ""

    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 7 -c tessedit_char_whitelist=0123456789')
    tess_text = tess_text.strip()

    candidates = [easy_text.strip(), tess_text]
    digits = [c for c in candidates if c.isdigit()]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)

# === Adım 5: Sonuçları görselleştirme ===
for box, number in zip(boxes, ocr_results):
    x1, y1, x2, y2 = box
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"📸 Çıktı kaydedildi: {output_path}")

# === OCR Sonuçlarını yazdır ve CSV olarak kaydet ===
print("\n🔍 OCR Sonuçları:")
for i, number in enumerate(ocr_results):
    print(f"Crop {i}: {number}")

ocr_df = pd.DataFrame({
    "crop_file": [f"crop_{i}.png" for i in range(len(ocr_results))],
    "predicted_number": ocr_results
})
ocr_df.to_csv("output/ocr_results.csv", index=False)
print("📄 OCR sonuçları CSV olarak kaydedildi: output/ocr_results.csv")

# === Son görseli göster ===
plt.figure(figsize=(16, 9))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti")
plt.show()

Using cache found in C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-25 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


🔁 Real-ESRGAN ile netleştirme başlatılıyor...
✅ Netleştirme tamamlandı. OCR işlemine geçiliyor...
📸 Çıktı kaydedildi: output\results_frame1.jpg

🔍 OCR Sonuçları:
Crop 0: ?
Crop 1: ?
Crop 2: ?
Crop 3: ?
Crop 4: ?
Crop 5: ?
Crop 6: ?
Crop 7: ?
Crop 8: ?
Crop 9: ?
Crop 10: ?
Crop 11: ?
Crop 12: ?
Crop 13: ?
Crop 14: ?
Crop 15: ?
Crop 16: ?
Crop 17: ?
Crop 18: ?
Crop 19: ?
Crop 20: ?
Crop 21: ?
Crop 22: ?
Crop 23: ?
📄 OCR sonuçları CSV olarak kaydedildi: output/ocr_results.csv


In [38]:
import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
import os
import subprocess
import pandas as pd
import re
from transformers import VisionEncoderDecoderModel, AutoProcessor

# === Yol Ayarları ===
image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")
crop_dir = Path("crops")
sr_crop_dir = Path("sr_crops")
output_path = Path("output/results_frame1.jpg")
output_path.parent.mkdir(exist_ok=True)
yolo_weights = "weights/yolov5s.pt"

crop_dir.mkdir(exist_ok=True)
sr_crop_dir.mkdir(exist_ok=True)

# === PARSeq OCR Yükleniyor ===
parseq_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
parseq_processor = AutoProcessor.from_pretrained("microsoft/trocr-base-stage1")
device = "cuda" if torch.cuda.is_available() else "cpu"
parseq_model.to(device).eval()

# === Kontrast Artırma Fonksiyonu ===
def enhance_contrast(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

# === YOLOv5 ile Oyuncu Tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Crop (Sırt Bölgesi) + Kontrast + Filtreleme ===
boxes, valid_crop_indices = [], []
for i, det in enumerate(preds):
    x1, y1, x2, y2, _, _ = map(int, det[:6])
    torso_y1 = y1 + int((y2 - y1) * 0.4)
    torso_y2 = y1 + int((y2 - y1) * 0.9)
    crop = img[torso_y1:torso_y2, x1:x2]

    if crop.size == 0:
        continue

    crop = enhance_contrast(crop)
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    blur = cv2.Laplacian(gray, cv2.CV_64F).var()
    contrast = gray.std()
    print(f"Crop {i}: blur={blur:.2f}, contrast={contrast:.2f}")

    if blur > 20 and contrast > 20:
        crop_path = crop_dir / f"crop_{i}.png"
        cv2.imwrite(str(crop_path), crop)
        boxes.append((x1, torso_y1, x2, torso_y2))
        valid_crop_indices.append(i)

# === Real-ESRGAN ile Netleştirme ===
print("🔁 Real-ESRGAN başlatılıyor...")
subprocess.run([
    "python", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", "crops",
    "-o", "sr_crops",
    "--fp32"
], capture_output=True, text=True)

print("✅ Netleştirme tamamlandı. OCR başlıyor...")

# === OCR Fonksiyonu (PARSeq) ===
def run_parseq(img_pil):
    inputs = parseq_processor(images=img_pil, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = parseq_model.generate(**inputs)
    return parseq_processor.batch_decode(outputs, skip_special_tokens=True)[0]

# === OCR İşlemi: EasyOCR + Tesseract + PARSeq ===
reader = easyocr.Reader(["en"], gpu=True)
ocr_results, ocr_logs = [], []

for j, i in enumerate(valid_crop_indices):
    box = boxes[j]
    sr_crop_path = sr_crop_dir / f"crop_{i}_out.png"
    if not sr_crop_path.exists():
        ocr_results.append("?")
        ocr_logs.append((i, "?", 0.0, "missing", "missing"))
        continue

    crop = cv2.imread(str(sr_crop_path))
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    # EasyOCR
    easy_out = reader.readtext(np.array(crop))
    easy_text, easy_conf = "", 0.0
    if easy_out:
        easy_text = easy_out[0][1]
        easy_conf = easy_out[0][2]
        if easy_conf < 0.4: easy_text = ""

    # Tesseract
    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 8 -c tessedit_char_whitelist=0123456789').strip()

    # PARSeq
    parseq_text = run_parseq(crop_pil)

    # Adaylar
    candidates = [easy_text.strip(), tess_text, parseq_text.strip()]
    digits = [re.findall(r'\d+', c) for c in candidates if c]
    digits = [d[0] for d in digits if d]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)
    ocr_logs.append((i, final_text, easy_conf, tess_text, parseq_text))

# === Orijinal Görsel Üzerine Yaz ===
for (x1, y1, x2, y2), number in zip(boxes, ocr_results):
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"📸 Çıktı kaydedildi: {output_path}")

# === OCR Detay Logları ve CSV ===
print("\n🔍 OCR Sonuçları (Detaylı):")
for i, number, conf, tess, parseq in ocr_logs:
    print(f"Crop {i}: Tahmin='{number}' | EasyOCR_conf={conf:.2f} | Tesseract='{tess}' | PARSeq='{parseq}'")

ocr_df = pd.DataFrame({
    "crop_file": [f"crop_{i}.png" for i in valid_crop_indices],
    "predicted_number": ocr_results
})
ocr_df.to_csv("output/ocr_results.csv", index=False)
print("📄 OCR sonuçları CSV olarak kaydedildi: output/ocr_results.csv")

# === Görseli Göster ===
plt.figure(figsize=(16, 9))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti")
plt.show()


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.50.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

Crop 0: blur=161.45, contrast=41.56
Crop 1: blur=172.76, contrast=43.02
Crop 2: blur=186.59, contrast=36.00
Crop 3: blur=190.06, contrast=31.45
Crop 4: blur=193.84, contrast=42.32
Crop 5: blur=140.03, contrast=33.31
Crop 6: blur=259.64, contrast=46.25
Crop 7: blur=376.03, contrast=37.57
Crop 8: blur=202.15, contrast=34.23
Crop 9: blur=304.95, contrast=24.38
Crop 10: blur=537.24, contrast=48.64
Crop 11: blur=385.01, contrast=44.56
Crop 12: blur=413.78, contrast=44.87
Crop 13: blur=12863.94, contrast=62.26
Crop 14: blur=323.14, contrast=43.50
Crop 15: blur=265.58, contrast=40.65
Crop 16: blur=238.08, contrast=46.78
Crop 17: blur=30715.64, contrast=65.45
Crop 18: blur=34926.11, contrast=68.17
Crop 19: blur=508.76, contrast=50.82
Crop 20: blur=382.73, contrast=36.18
Crop 21: blur=1254.65, contrast=52.06
Crop 22: blur=375.23, contrast=36.04
Crop 23: blur=505.84, contrast=40.28
🔁 Real-ESRGAN başlatılıyor...
✅ Netleştirme tamamlandı. OCR başlıyor...


ValueError: You need to specify either `text` or `text_target`.

In [39]:
!pip uninstall torch torchvision torchaudio -y


Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torchaudio 2.6.0+cu118
Uninstalling torchaudio-2.6.0+cu118:
  Successfully uninstalled torchaudio-2.6.0+cu118


In [45]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118

^C


In [37]:
!pip install torchvision --force-reinstall


  Using cached torchvision-0.21.0-cp39-cp39-win_amd64.whl.metadata (6.3 kB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached torch-2.6.0-cp39-cp39-win_amd64.whl.metadata (28 kB)
  Using cached pillow-11.1.0-cp39-cp39-win_amd64.whl.metadata (9.3 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp39-cp39-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------------- ------------------- 0.8/1.6 MB 3.4 MB/s eta 0:00:01
   --------------------------------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu118 requires torch==2.6.0+cu118, but you have torch 2.6.0 which is incompatible.


In [35]:
import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from PIL import Image
from pathlib import Path
from matplotlib import pyplot as plt
import os
import subprocess
import pandas as pd
import re
from transformers import VisionEncoderDecoderModel, AutoProcessor
import torchvision.transforms as T

# === Yol Ayarları ===
image_path = Path("Ekran görüntüsü 2025-03-31 152531.png")
crop_dir = Path("crops")
sr_crop_dir = Path("sr_crops")
output_path = Path("output/results_frame1.jpg")
output_path.parent.mkdir(exist_ok=True)
yolo_weights = "weights/yolov5s.pt"

crop_dir.mkdir(exist_ok=True)
sr_crop_dir.mkdir(exist_ok=True)

# === PARSeq Yükle ===
parseq_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
parseq_processor = AutoProcessor.from_pretrained("microsoft/trocr-base-stage1")
parseq_model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# === Super Resolution için kontrastı artırma ===
def enhance_contrast(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

# === YOLOv5 ile Oyuncu Tespiti ===
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_weights)
model.conf = 0.25

img = cv2.imread(str(image_path))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb)
preds = results.xyxy[0].cpu().numpy()

# === Üst Gövde Crop & Kontrast Artırımı ===
boxes, valid_crop_indices = [], []
for i, det in enumerate(preds):
    x1, y1, x2, y2, _, _ = map(int, det[:6])
    torso_y1 = y1 + int((y2 - y1) * 0.4)
    torso_y2 = y1 + int((y2 - y1) * 0.9)
    crop = img[torso_y1:torso_y2, x1:x2]
    
    if crop.size == 0: continue
    crop = enhance_contrast(crop)

    blur = cv2.Laplacian(cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()
    contrast = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY).std()
    print(f"Crop {i}: blur={blur:.2f}, contrast={contrast:.2f}")

    if blur > 20 and contrast > 20:
        crop_path = crop_dir / f"crop_{i}.png"
        cv2.imwrite(str(crop_path), crop)
        boxes.append((x1, torso_y1, x2, torso_y2))
        valid_crop_indices.append(i)

# === Real-ESRGAN ile Netleştirme ===
print("🔁 Real-ESRGAN başlatılıyor...")
subprocess.run([
    "python", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", "crops",
    "-o", "sr_crops",
    "--fp32"
], capture_output=True, text=True)

print("✅ Netleştirme tamamlandı. OCR başlıyor...")

# === OCR Fonksiyonu (PARSeq) ===
def run_parseq(img_pil):
    inputs = parseq_processor(images=img_pil, return_tensors="pt").to(parseq_model.device)
    with torch.no_grad():
        outputs = parseq_model.generate(**inputs)
    return parseq_processor.batch_decode(outputs, skip_special_tokens=True)[0]

# === OCR İşlemi ===
reader = easyocr.Reader(["en"], gpu=True)
ocr_results, ocr_logs = [], []

for j, i in enumerate(valid_crop_indices):
    box = boxes[j]
    sr_crop_path = sr_crop_dir / f"crop_{i}_out.png"
    if not sr_crop_path.exists():
        ocr_results.append("?")
        ocr_logs.append((i, "?", 0.0, "missing", "missing"))
        continue

    crop = cv2.imread(str(sr_crop_path))
    crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))

    easy_out = reader.readtext(np.array(crop))
    easy_text, easy_conf = "", 0.0
    if easy_out:
        easy_text = easy_out[0][1]
        easy_conf = easy_out[0][2]
        if easy_conf < 0.4: easy_text = ""

    tess_text = pytesseract.image_to_string(crop_pil, config='--psm 8 -c tessedit_char_whitelist=0123456789').strip()
    parseq_text = run_parseq(crop_pil)

    candidates = [easy_text.strip(), tess_text, parseq_text.strip()]
    digits = [re.findall(r'\d+', c) for c in candidates if c]
    digits = [d[0] for d in digits if d]
    final_text = digits[0] if digits else "?"

    ocr_results.append(final_text)
    ocr_logs.append((i, final_text, easy_conf, tess_text, parseq_text))

# === Görsel Üzerine Yaz ===
for (x1, y1, x2, y2), number in zip(boxes, ocr_results):
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite(str(output_path), img)
print(f"📸 Çıktı kaydedildi: {output_path}")

# === OCR Sonuçları Log ve CSV ===
print("\n🔍 OCR Sonuçları (Detaylı):")
for i, number, conf, tess, parseq in ocr_logs:
    print(f"Crop {i}: Tahmin='{number}' | EasyOCR_conf={conf:.2f} | Tesseract='{tess}' | PARSeq='{parseq}'")

ocr_df = pd.DataFrame({
    "crop_file": [f"crop_{i}.png" for i in valid_crop_indices],
    "predicted_number": ocr_results
})
ocr_df.to_csv("output/ocr_results.csv", index=False)

# === Görseli Göster ===
plt.figure(figsize=(16, 9))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Forma Numarası Tespiti")
plt.show()


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.50.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

Crop 0: blur=161.45, contrast=41.56
Crop 1: blur=172.76, contrast=43.02
Crop 2: blur=186.59, contrast=36.00
Crop 3: blur=190.06, contrast=31.45
Crop 4: blur=193.84, contrast=42.32
Crop 5: blur=140.03, contrast=33.31
Crop 6: blur=259.64, contrast=46.25
Crop 7: blur=376.03, contrast=37.57
Crop 8: blur=202.15, contrast=34.23
Crop 9: blur=304.95, contrast=24.38
Crop 10: blur=537.24, contrast=48.64
Crop 11: blur=385.01, contrast=44.56
Crop 12: blur=413.78, contrast=44.87
Crop 13: blur=12863.94, contrast=62.26
Crop 14: blur=323.14, contrast=43.50
Crop 15: blur=265.58, contrast=40.65
Crop 16: blur=238.08, contrast=46.78
Crop 17: blur=30715.64, contrast=65.45
Crop 18: blur=34926.11, contrast=68.17
Crop 19: blur=508.76, contrast=50.82
Crop 20: blur=382.73, contrast=36.18
Crop 21: blur=1254.65, contrast=52.06
Crop 22: blur=375.23, contrast=36.04
Crop 23: blur=505.84, contrast=40.28
🔁 Real-ESRGAN başlatılıyor...
✅ Netleştirme tamamlandı. OCR başlıyor...


ValueError: You need to specify either `text` or `text_target`.

In [34]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())


2.6.0+cu118
0.21.0+cu118
True


In [13]:
from glob import glob
print(glob("crops/*.png"))


['crops\\crop_0.png', 'crops\\crop_1.png', 'crops\\crop_10.png', 'crops\\crop_11.png', 'crops\\crop_12.png', 'crops\\crop_13.png', 'crops\\crop_14.png', 'crops\\crop_15.png', 'crops\\crop_16.png', 'crops\\crop_17.png', 'crops\\crop_18.png', 'crops\\crop_19.png', 'crops\\crop_2.png', 'crops\\crop_20.png', 'crops\\crop_21.png', 'crops\\crop_22.png', 'crops\\crop_23.png', 'crops\\crop_3.png', 'crops\\crop_4.png', 'crops\\crop_5.png', 'crops\\crop_6.png', 'crops\\crop_7.png', 'crops\\crop_8.png', 'crops\\crop_9.png']


In [22]:
result = subprocess.run([
    "python", "Real-ESRGAN/inference_realesrgan.py",
    "-n", "RealESRGAN_x4plus",
    "-i", "crops",
    "-o", "sr_crops",
    "--fp32"
], capture_output=True, text=True)


print(result.stdout)
print(result.stderr)


Testing 0 crop_0
Testing 1 crop_1
Testing 2 crop_10
Testing 3 crop_11
Testing 4 crop_12
Testing 5 crop_13
Testing 6 crop_14
Testing 7 crop_15
Testing 8 crop_16
Testing 9 crop_17
Testing 10 crop_18
Testing 11 crop_19
Testing 12 crop_2
Testing 13 crop_20
Testing 14 crop_21
Testing 15 crop_22
Testing 16 crop_23
Testing 17 crop_3
Testing 18 crop_4
Testing 19 crop_5
Testing 20 crop_6
Testing 21 crop_7
Testing 22 crop_8
Testing 23 crop_9




In [ ]:
import torch
print("CUDA kullanılabilir mi? 👉", torch.cuda.is_available())
print("Aktif cihaz:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Yok")


In [15]:
!pip install basicsr


  Using cached basicsr-1.4.2-py3-none-any.whl
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached yapf-0.43.0-py3-none-any.whl.metadata (46 kB)
  Using cached absl_py-2.2.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached grpcio-1.71.0-cp39-cp39-win_amd64.whl.metadata (4.0 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached protobuf-6.30.2-cp39-cp39-win_amd64.whl.metadata (593 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
Using cached future-1.0.0-py3-none-any.whl (491 kB)
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.5 MB ? eta -:--:--
   ----------- ---------------------------- 1.6/5.5 MB 7.6 MB/s eta 0:00:01
   ------------------- -------

In [17]:
!pip install -r Real-ESRGAN/requirements.txt


  Using cached facexlib-0.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached gfpgan-1.3.8-py3-none-any.whl.metadata (12 kB)
  Using cached filterpy-1.4.5-py3-none-any.whl
  Using cached numba-0.60.0-cp39-cp39-win_amd64.whl.metadata (2.8 kB)
  Using cached llvmlite-0.43.0-cp39-cp39-win_amd64.whl.metadata (4.9 kB)
Using cached facexlib-0.3.0-py3-none-any.whl (59 kB)
Using cached gfpgan-1.3.8-py3-none-any.whl (52 kB)
Using cached numba-0.60.0-cp39-cp39-win_amd64.whl (2.7 MB)
Using cached llvmlite-0.43.0-cp39-cp39-win_amd64.whl (28.1 MB)


In [20]:
!pip install torchvision==0.14.1


   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/162.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/162.5 MB 3.0 MB/s eta 0:00:54
   ---------------------------------------- 1.8/162.5 MB 4.0 MB/s eta 0:00:40
    --------------------------------------- 3.7/162.5 MB 5.7 MB/s eta 0:00:28
   - -------------------------------------- 6.3/162.5 MB 7.3 MB/s eta 0:00:22
   -- ------------------------------------- 8.7/162.5 MB 8.1 MB/s eta 0:00:19
   -- ------------------------------------- 11.0/162.5 MB 8.6 MB/s eta 0:00:18
   --- ------------------------------------ 12.8/162.5 MB 8.8 MB/s eta 0:00:18
   --- ------------------------------------ 14.9/162.5 MB 8.7 MB/s eta 0:00:17
   ---- ----------------------------------- 17.0/162.5 MB 8.9 MB/s eta 0:00:17
  

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.1 requires torch>=2.1.0, but you have torch 1.13.1 which is incompatible.
torchaudio 2.6.0+cu118 requires torch==2.6.0+cu118, but you have torch 1.13.1 which is incompatible.
torchmetrics 1.7.0 requires torch>=2.0.0, but you have torch 1.13.1 which is incompatible.


In [1]:
import cv2
import numpy as np
import os

output_dir = "number_dataset"
os.makedirs(output_dir, exist_ok=True)

fonts = [cv2.FONT_HERSHEY_SIMPLEX, cv2.FONT_HERSHEY_COMPLEX, cv2.FONT_HERSHEY_DUPLEX]

for num in range(1, 101):
    label = str(num)
    label_dir = os.path.join(output_dir, label)
    os.makedirs(label_dir, exist_ok=True)
    
    for i in range(100):  # Her sayıdan 100 tane üret
        img = np.ones((64, 64, 3), dtype=np.uint8) * 255
        font = np.random.choice(fonts)
        font_scale = np.random.uniform(0.8, 1.2)
        thickness = np.random.randint(1, 3)
        text_size = cv2.getTextSize(label, font, font_scale, thickness)[0]
        text_x = (img.shape[1] - text_size[0]) // 2
        text_y = (img.shape[0] + text_size[1]) // 2
        cv2.putText(img, label, (text_x, text_y), font, font_scale, (0, 0, 0), thickness)
        cv2.imwrite(f"{label_dir}/{i}.png", img)


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(100, activation='softmax')  # 100 sınıf için
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = train_datagen.flow_from_directory(
    'number_dataset',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    'number_dataset',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

model.fit(train_gen, validation_data=val_gen, epochs=10)


In [4]:
!pip install tensorflow-gpu


  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [39 lines of output]
  Traceback (most recent call last):
    File "C:\Users\uysal\anaconda3\envs\jersey-env-2\lib\site-packages\packaging\requirements.py", line 36, in __init__
      parsed = _parse_requirement(requirement_string)
    File "C:\Users\uysal\anaconda3\envs\jersey-env-2\lib\site-packages\packaging\_parser.py", line 62, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
    File "C:\Users\uysal\anaconda3\envs\jersey-env-2\lib\site-packages\packaging\_parser.py", line 80, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
    File "C:\Users\uysal\anaconda3\envs\jersey-env-2\lib\site-packages\packaging\_parser.py", line 124, in _parse_requirement_details
      marker = _parse_requirement_marker(
    File "C:\Users\uysal\anaconda3\envs\jersey-env-2\lib\site-packages\packaging\_parser